# To-do List
# Abstract
According to the course requirements, we are tasked with building an LLM application based on the Reflection Module.

Therefore, in this assignment, I will simulate a real-world scenario to enhance a RAG-based Threat Intelligence Security Chatbot using the Reflection Module.

# 1. Introduction
## 1.1. Reflection Module
The Reflection Module consists of three key stages:

Firstly, it processes the user's input using an LLM.

Secondly, it uses the LLM again to refine the initial output.

Finally, it generates the improved result.

## 1.2. Enhancing RAG Applications with the Reflection Module
In cybersecurity scenarios—whether in threat intelligence reports or online articles—researchers often use highly technical or obscure terminology. This makes it difficult to directly map a user's input to relevant documents in a Threat Intelligence Database using vector search alone, often leading to misinterpretations.

To address this, we propose using the Reflection Module to pre-process and refine user input. By transforming non-expert language into more professional and domain-specific terms, we aim to improve the relevance and accuracy of vector search results.

# 2. Development
Based on the aforementioned considerations,
In this assignment for the course "AI Agent: Reflection Module", I will develop a security chatbot with the following features:

Reflection Module-enhanced Chatbot

Threat Intelligence-aware Chatbot

# Reflection Module Chatbot

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
reflection_client = OpenAI(
    api_key = os.environ.get("OPENAI_API_KEY")
)


def reply(system_prompt,
          user_prompt
          ):


    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    chat_completion = reflection_client.chat.completions.create(
        model= "gpt-4o-mini", # save money
        messages= messages,
        max_tokens= 500 # cot very expensive must be limit output token.
    )


    reply = chat_completion.choices[0].message.content

    return reply

def reflect_post(user_prompt):
    # Step 1: Writer 初稿
    system_writer = "你是一個優秀的資安分析顧問，請幫助使用者梳理情境，將使用者較不專業的言語轉換為適合用於威脅情資的版本"
    first_version = reply(system_prompt= system_writer, 
                          user_prompt = user_prompt
                          )


    # Step 2: Reviewer 給建議
    system_reviewer = "你是一位資安文案潤稿專家，擅長分析一篇威脅情資是否足夠專業，並給出建議。"
    suggestion = reply(system_prompt= system_reviewer, 
                        user_prompt = first_version
                        )


    # Step 3: Writer 再寫一次（根據建議）
    second_system_prompt = "你是個優化的資安顧問，善於根據修改意見優化威脅情資的結構"
    second_prompt = f"這是我剛剛寫的貼文：\n{first_version}\n\n這是修改建議：\n{suggestion}\n\n請根據這些建議，幫我改得更生活化、更自然。請用台灣習慣的中文, 並且只要輸出改好的文章就可以了。"

    second_version = reply(system_prompt= second_system_prompt, 
                            user_prompt = second_prompt
                            )
    
    return first_version, suggestion, second_version



In [3]:
import gradio
with gradio.Blocks() as demo:
    gradio.Markdown("### 🤖 資安威脅描述優化（Reflection Agent）")
    user_input = gradio.Textbox(label="請輸入你今天發現的資安事件")
    btn = gradio.Button("生成貼文 & 修正建議")

    with gradio.Row():
        out1 = gradio.Textbox(label="🌟 第一版貼文 (model_writer)")
        out2 = gradio.Textbox(label="🧐 修改建議 (model_reviewer)")
        out3 = gradio.Textbox(label="✨ 第二版貼文 (model_writer 改寫)")

    btn.click(reflect_post, inputs=[user_input], outputs=[out1, out2, out3])

demo.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Keyboard interruption in main thread... closing server.


In [ ]:
user_prompt = '''
    My desktop is showing a 24-hour countdown, demanding that I pay in Bitcoin to unlock it, or else my data will be stolen.

    I just discovered a strange folder placed in the root directory of my Windows system. It's named "ysytem32
    '''

# Threat Intellgence Chatbot

In [ ]:
# Embedding
import pandas
import tqdm as notebook_tqdm

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document 

from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(
    api_key = os.environ.get("OPENAI_API_KEY")
)

# cot prompt + RAG
system_prompt_templete = '''
    你是一個優秀的資安分析師，現在需要幫助使用者分辨當前的情境屬於何種資安威脅。並參考威脅情資內容後，根據以下內容 Step by Step 分析。
    參考情資與使用者情境分析，
    1. 使用者可能是如何被入侵的
    2. 使用者目前可以先做何種緩解措施
'''

# RAG Post
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")
vectorstore = FAISS.load_local(
    "data/faiss_db",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

messages = [
            {"role": "system", "content": system_prompt_templete}
            ]

import gradio
with gradio.Blocks() as demo:
    gradio.Markdown("# Professional Security Consulting")
    chatbot = gradio.Chatbot(type="messages")
    msg = gradio.Textbox(placeholder="請輸入你的問題...")
    state = gradio.State(messages)

    def main_chatbot(user_prompt, messages):
        results = vectorstore.similarity_search(user_prompt, k=1)
        RAG_Post = results[0].page_content
        user_prompt_templete = '''
            使用者情境
                {user_prompt}
            威脅情資  
                {RAG_Post}
        '''

        messages.append({"role": "user", "content": user_prompt_templete.format(user_prompt = user_prompt, RAG_Post= RAG_Post)})

        chat_completion = client.chat.completions.create(
            model= "gpt-4o-mini", # save money
            messages= messages,
            max_tokens= 500 # cot very expensive must be limit output token.
        )

        reply = chat_completion.choices[0].message.content
        messages.append({"role": "assistant", "content": reply}) # 透過添加歷史對話紀錄，變相讓 LLM 記得說了些什麼。改成這個寫法之後要 debug 也會比較方便。

        return "", messages, messages

    msg.submit(
            fn=main_chatbot,
            inputs=[msg, state],
            outputs=[msg, chatbot, state]
        )

demo.launch(share=True, debug=True)